In [1]:
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="llava")

In [142]:
meeting_details ="""
1: Item 11. Recommendation to authorize city manager to execute all documents necessary with Friends of Colorado Lagoon for the continued tendency and operations of Colorado Lagoon, Wetland and Marine Science Education Center and authorize city manager to execute all documents necessary.
1: For Third Amendment right of entry permit number 3 to 9 three with friends of Colorado Lagoon from Parks and Rec District three.
0: Yes, that's one question.
5: Putting the effort that they put into this lease agreement. I want to acknowledge very publicly and very enthusiastically the great work that the Friends of Colorado Lagoon does for our city, the efforts that they have been involved in for a number of years, many, many years prior to me getting on council and with my predecessor and with
5: his predecessor has just been tremendous. Through their great efforts and partnership with the city of the Colorado Lagoon, water quality has been restored to a place where we can be very proud of getting a ratings year after year.
5: And that has a lot to do with the advocacy and the passion of SoCal. So I'm happy to support this lease agreement. I did want to call out one thing, and that's an amendment to the lease that's in here.
5: And I want to thank staff for putting it in here, but it basically highlights the focus of the city, which is to work collaboratively with our nonprofits, not work competitively with them, to encourage them to apply for grants to support them in their grant funding effort, and to also allow the city to analyze whether those grant funding
5: opportunities are going to encumber the city and if they are whether it's it's feasible for the city to support such endeavors. And so.
3: I'm grateful for.
5: The language on page three that has been and added to reflect some of my concerns in that regards. And I want to thank staff for working collaboratively with my office and with SoCal to make that happen. So thank you.
5: Meredith Reynolds has been our lead on this. I appreciate your service. Thank you.
0: Thank you, Councilman Price. Now we're going to go into any comments on this, the item, any public comments.
1: There's no public comment on this item.
0: We're fine. Thank you very much. We don't have to vote on it either.
1: We do.
0: Okay, fine. See? No further comment or digest. Would you please cast your vote?
1: Council memo appears. But she carries.
0: Thank you. Now we have a special presentation and. I saw a presentation tonight. And that presentation and the Peacemaker celebration tonight, the peacemakers presentation were honored. Some great individuals, as some of you may already know, each year after our annual Martin Luther King parade and celebration, I would like for us to honor a few individuals in the
"""

template = """
## Meeting Minutes

**Date:** [Date]
**Time:** [Start Time] - [End Time]
**Location:** [Location]
**Attendees:**
- [Person 1]
- [Person 2]
- [Person 3]

### Agenda
1. [Agenda Item 1]
2. [Agenda Item 2]
3. [Agenda Item 3]

### Discussion Summary
- **[Agenda Item 1]**: [Discussion Points]
- **[Agenda Item 2]**: [Discussion Points]
- **[Agenda Item 3]**: [Discussion Points]

### Action Items
- [Action Item 1]: [Person Responsible] - [Deadline]
- [Action Item 2]: [Person Responsible] - [Deadline]
- [Action Item 3]: [Person Responsible] - [Deadline]

### Decisions Made
- [Decision 1]
- [Decision 2]
- [Decision 3]

**Next Meeting:**
- **Date:** [Next Meeting Date]
- **Time:** [Next Meeting Time]
- **Location:** [Next Meeting Location]

**Minutes Prepared by:**
[Your Name]
"""


prompt = f"""
I’m Meeting summary officer pls help me to Summarize using meeting minute template 
We have finished the meeting:

{meeting_details}

use this template:

{template}

Do not enter information that is not displayed.
Do not include information that does not appear in the meeting like date, Location, Time but if it have fill .
Summarize using meeting minute template Only
"""

In [143]:
def generate_summary(prompt):
    prompt = f"""{prompt}"""
    result = llm.generate([prompt])
    summary = result.generations[0][0].text
    return summary

In [144]:
def generate_questions(meeting_details):
    prompt = f"""
    Based on the following meeting details, generate questions to evaluate the accuracy of the summary. The questions should be based on a meeting minutes template and should be in one of the following formats: yes/no questions, short answers, names only, or numbers only.
    i need 10 questions
    Do not generate awsnwer generate question only
    
    Meeting Details:
    {meeting_details}

    Questions is:
    """
    result = llm.generate([prompt])
    questions = result.generations[0][0].text.split("\n")
    return questions


In [145]:
generate_questions(meeting_details)

[' 1. Was an authorization requested for the city manager to execute all necessary documents with Friends of Colorado Lagoon?',
 '2. Was a right of entry permit number 3 to 9 issued to Friends of Colorado Lagoon from Parks and Rec District three?',
 '3. Was public acknowledgement given for the great work that Friends of Colorado Lagoon does for the city?',
 '4. Was there mention of the lease agreement with Friends of Colorado Lagoon and the amendment to the lease?',
 '5. Were any specific concerns regarding collaboration with nonprofits mentioned in the meeting?',
 '6. Was there a presentation honoring individuals at the Peacemaker celebration?',
 '7. Was there any public comment on the item being discussed?',
 '8. Was a vote taken on the item being discussed?',
 '9. What was the outcome of the vote, if taken?',
 '10. Were there any special presentations during the meeting? ']

In [146]:
def extract_answers(summary, questions):
    answers = []
    for question in questions:
        prompt = f"""
        Based on the following summary of the meeting minutes, answer the question briefly. Use only yes/no, short answers, names, or numbers. 
        Do not provide explanations.
        Do not explain the answer

        Summary:
        {summary}

        Question:
        {question}

        Answer:
        """
        result = llm.generate([prompt])
        answer = result.generations[0][0].text.strip()
        answers.append(answer)
    return answers


In [147]:
def compare_answers(source_answers, summary_answers):
    consistency_score = 0
    for source, summary in zip(source_answers, summary_answers):
        if source.lower() == summary.lower():
            consistency_score += 1
    return round(consistency_score / len(source_answers), 2)

In [148]:
def evaluate_summary_factual_consistency(meeting_details, prompt, max_iterations, template):
    questions = generate_questions(meeting_details)
    
    best_consistency_score = 0
    best_summary = None
    best_source_answers = None
    best_summary_answers = None
    
    iterations = 0
    
    while best_consistency_score < 0.6 and iterations < max_iterations:
        summary = generate_summary(prompt)
        source_answers = extract_answers(meeting_details, questions)
        summary_answers = extract_answers(summary, questions)
        consistency_score = compare_answers(source_answers, summary_answers)
        
        print(f"Iteration {iterations + 1}, Consistency Score: {consistency_score}")

        if consistency_score > best_consistency_score:
            best_consistency_score = consistency_score
            best_summary = summary
            best_source_answers = source_answers
            best_summary_answers = summary_answers

        if best_consistency_score < 0.6:
            prompt = f"""
            I’m Meeting summary officer pls help me to Summarize using meeting minute template 
            We have finished the meeting:
            
            {meeting_details}
            
            use this template:
            
            {template}
            
            Do not enter information that is not displayed.
            
            Make sure the summary can accurately answer these questions:
            
            {questions}
            """

        iterations += 1

    return {
        "summary": best_summary,
        "questions": questions,
        "source_answers": best_source_answers,
        "summary_answers": best_summary_answers,
        "consistency_score": best_consistency_score,
        "iterations": iterations
    }


In [149]:
evaluation_result = evaluate_summary_factual_consistency(meeting_details, prompt, 5, template)

Iteration 1, Consistency Score: 0.7


In [150]:
print("Generated Summary:")
print(evaluation_result["summary"])
print("\nGenerated Questions:")
print(evaluation_result["questions"])
print("\nSource Answers:")
print(evaluation_result["source_answers"])
print("\nSummary Answers:")
print(evaluation_result["summary_answers"])
print("\nConsistency Score:")
print(evaluation_result["consistency_score"])

Generated Summary:
 **Date:** [Insert Date]
**Time:** [Start Time] - [End Time]
**Location:** [Insert Location]
**Attendees:**
- Meredith Reynolds
- Councilman Price

### Agenda
1. Recommendation to authorize city manager to execute all documents necessary with Friends of Colorado Lagoon for the continued tendency and operations of Colorado Lagoon, Wetland and Marine Science Education Center and authorize city manager to execute all documents necessary.
2. For Third Amendment right of entry permit number 3 to 9 three with friends of Colorado Lagoon from Parks and Rec District three.

### Discussion Summary
- Councilman Price expressed his appreciation for the efforts of Friends of Colorado Lagoon in restoring water quality and acknowledged their advocacy and passion.
- He also requested an amendment to the lease agreement that highlights the focus of the city, which is to work collaboratively with nonprofits, not competitively with them, and to encourage them to apply for grants to sup